In [1]:
import tensorflow as tf
import numpy as np
tf.__version__

'2.0.0'

VGG块的组成规律是：连续使用数个相同的填充为1、窗口形状为 3×3 的卷积层后接上一个步幅为2、窗口形状为 2×2 的最大池化层。卷积层保持输入的高和宽不变，而池化层则对其减半。我们使用vgg_block函数来实现这个基础的VGG块，它可以指定卷积层的数量num_convs和输出通道数num_channels。

In [2]:
def vgg_block(num_convs,num_chanels):
    block = tf.keras.Sequential()
    
    for i in range(num_convs):
        block.add(tf.keras.layers.Conv2D(num_chanels,padding="SAME",kernel_size=3,activation="relu"))
    
    block.add(tf.keras.layers.MaxPooling2D(strides=(2,2)))
    
    return block

In [3]:
block = vgg_block(3,10)

In [8]:
x = tf.random.normal(shape=(10,10,3))
x = tf.expand_dims(x,axis=0)
x.shape

TensorShape([1, 10, 10, 3])

In [9]:
block(x)

<tf.Tensor: id=117, shape=(1, 5, 5, 10), dtype=float32, numpy=
array([[[[0.        , 0.10137452, 0.        , 0.14878502, 0.18936999,
          0.02584816, 0.09371764, 0.186245  , 0.02967511, 0.04456124],
         [0.        , 0.06417146, 0.        , 0.42978048, 0.30050197,
          0.13600703, 0.26666224, 0.11881069, 0.        , 0.22648938],
         [0.        , 0.03752999, 0.00922242, 0.5127882 , 0.32265666,
          0.0049084 , 0.33574885, 0.31328893, 0.        , 0.26153153],
         [0.        , 0.        , 0.        , 0.6435751 , 0.6219014 ,
          0.2828443 , 0.40782544, 0.52491957, 0.        , 0.5629289 ],
         [0.45518944, 0.1427342 , 0.27428615, 0.16438232, 0.5126835 ,
          0.18664551, 0.49226454, 0.55698943, 0.23810351, 0.1609131 ]],

        [[0.        , 0.0620217 , 0.        , 0.33673525, 0.24686882,
          0.        , 0.3010417 , 0.18256554, 0.2220302 , 0.10178997],
         [0.06329435, 0.19489193, 0.        , 0.5316355 , 0.03040163,
          0.0771538

In [10]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [18]:
def create_model(conv_arch,input_shape):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape= input_shape))
    for num_conv,num_chanels in conv_arch:
        model.add(vgg_block(num_conv,num_chanels))
        
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(4096,activation="relu"))
    model.add(tf.keras.layers.Dense(4096,activation="relu"))
    model.add(tf.keras.layers.Dense(10,activation="softmax"))
    return model

In [19]:
model = create_model(conv_arch,input_shape=(224,224,3))

In [20]:
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,metrics=["acc"])

In [22]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_8 (Sequential)    (None, 112, 112, 64)      1792      
_________________________________________________________________
sequential_9 (Sequential)    (None, 56, 56, 128)       73856     
_________________________________________________________________
sequential_10 (Sequential)   (None, 28, 28, 256)       885248    
_________________________________________________________________
sequential_11 (Sequential)   (None, 14, 14, 512)       3539968   
_________________________________________________________________
sequential_12 (Sequential)   (None, 7, 7, 512)         4719616   
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 4096)             

In [23]:
x = tf.random.normal(shape=(224,224,3))
x = tf.expand_dims(x,axis=0)
x.shape

TensorShape([1, 224, 224, 3])

In [24]:
model(x)

<tf.Tensor: id=1017, shape=(1, 10), dtype=float32, numpy=
array([[0.10672276, 0.09923742, 0.09782732, 0.09883187, 0.09900384,
        0.09615097, 0.09905155, 0.10240912, 0.0993533 , 0.10141183]],
      dtype=float32)>

In [25]:
IMG_WIDTH = 224
IMG_HEIGHT = 224
BATCH_SIZE = 5

In [26]:
DATA_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(fname="flower_photos",origin=DATA_URL,untar=True)
import pathlib
data_dir = pathlib.Path(data_dir)
ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))
class_lable =  np.array([ item.name for item in data_dir.glob("*") if item.name != "LICENSE.txt" ])
## 转换lable为数值型数据
lable_dic =dict( zip(class_lable,np.asarray(range(0,len(class_lable)))))
@tf.function
def get_lable(x):
    arr_str = tf.strings.split(x,sep="\\")
    return arr_str[-2] == class_lable

def decode_img(img):
    img = tf.image.decode_image(img,channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize_with_crop_or_pad(img,IMG_WIDTH,IMG_HEIGHT)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    return img

def img_map_fun(x):
    lable = get_lable(x)
    img = tf.io.read_file(x)
    img = decode_img(img)
    return img,lable

lable = tf.constant([0,1,2,3,4])
@tf.function
def map_lable_fun(x,y):
    print(y)
    y = lable[y][0]
    return x,y

ds = ds.map(map_func=img_map_fun,num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds = ds.map(map_func=map_lable_fun)

def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000,batch_size = 256):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
    else:
        ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(batch_size)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

ds = prepare_for_training(ds,batch_size = BATCH_SIZE)

image_batch, label_batch = next(iter(ds))

Tensor("y:0", shape=(5,), dtype=bool)


In [27]:
model(image_batch)

<tf.Tensor: id=1589, shape=(5, 10), dtype=float32, numpy=
array([[0.10124978, 0.09954045, 0.09985214, 0.09997739, 0.09965686,
        0.09940214, 0.1002736 , 0.10035679, 0.09943722, 0.10025369],
       [0.10150246, 0.09910905, 0.09979012, 0.09949955, 0.09971195,
        0.09915818, 0.10021252, 0.1006892 , 0.09997004, 0.10035697],
       [0.10143833, 0.09951892, 0.09982795, 0.10009196, 0.09982649,
        0.09937846, 0.10010269, 0.10006729, 0.09961801, 0.10012981],
       [0.10121918, 0.09976923, 0.09969419, 0.09999774, 0.09983264,
        0.0993958 , 0.10007992, 0.10049582, 0.09968666, 0.09982888],
       [0.10169157, 0.09927814, 0.09972777, 0.09979265, 0.09991218,
        0.09926024, 0.10000106, 0.10059597, 0.09956744, 0.100173  ]],
      dtype=float32)>

In [28]:
model.fit(ds,steps_per_epoch=5,epochs=2)

Train for 5 steps
Epoch 1/2
5/5 [==============================] - 52s 10s/step - loss: 11966.3771 - acc: 0.3200
Epoch 2/2
5/5 [==============================] - 40s 8s/step - loss: 8.9566 - acc: 0.2000
